## 機械学習入門講座 
今回の講座では主に下記の項目にフォーカスしています
- 機械学習を用いた文章間 類似度計算手法について、基礎的な仕組みを実際 データを分析することで体系的に理解する
- ハンズオン形式で実施することで、機械学習・数学初心者でも手法 大まかな概要を理解することができる
- Webサービスと機械学習手法の関わり方や機械学習 基本的な考え方を知ることで、受講者 スキルアップ・サービスへ 応用へ きっかけを提供する

このチュートリアルでは、実際のデータを用いてコーディング・類似度計算を行うことで実践力をつけることを目的とします。今回はグノシーに実際に掲載されている記事データを使った文章間の類似度計算及びトピックモデルに関する実験を行います。

## 1. 文章間の類似度計算
### 目的
pythonの機械学習ライブラリの一つである「gensim」を使って、以下について体系的に学びます
- Pythonによるデータ整形・分析処理について
- word2vecを用いた文章間の類似度計算について
- 類似度計算結果の可視化及びサービスへの応用について

### 使用するデータ
Web版グノシーに掲載されている記事データをクローリングし取得<br>
クローリング方法については弊社のデータ分析ブログに細かく載っていますのでぜひご覧下さい<br>
([Scrapy + Scrapy Cloudで快適Pythonクロール+スクレイピングライフを送る](http://data.gunosy.io/entry/python-scrapy-scraping))

### 講師
株式会社Gunosy 開発本部 データ分析部 荻原 崇 (FB: [ogiogi93](https://www.facebook.com/ogiogi93))<br>
開発本部データ分析部所属。2017年度新卒入社。大学院ではセンサを用いた人々の流動分析およびJR東日本との共同プロジェクトにて、電車の乗車率予測モデルに関する研究を行う。現在は動画配信プロジェクトのロジック、データ分析全般を担当。調理師免許保持者。

### アジェンダ
1. バックグラウンド
2. ライブラリのimportとデータの準備
3. 類似度計算を実施するためのデータの整形
4. word2vecによるモデルの生成と単語間での類似度計算
5. doc2vecによるモデルの生成と文章間での類似度計算

## 1. バックグラウンド
近年, 機械学習などによるデータ分析に注目が集まり、既に多くの業界・サービスに応用されています。数ある機械学習手法の中でも、word2vecやLDAは「検索・レコメンド・評判分析」など様々なタスクに応用することができます。そのため、これらの手法をまず身につけることで様々なタスク・領域で機械学習を応用することが可能となります。

In [1]:
from IPython.display import Image
Image(url='https://media.accel-brain.com/wp-content/uploads/2016/03/linear-relationships.png#link2keyword=%3Cimg%20class%3D%22aligncenter%20size-medium%20wp-image-951%22%20src%3D%22%2F%2Fmedia.accel-brain.com%2Fwp-content%2Fuploads%2F2016%2F03%2Flinear-relationships-500x175.png%22%20alt%3D%22linear-relationships%22%20width%3D%22500%22%20height%3D%22175%22%20%2F%3E')

## 2. ライブラリのインポートとデータの準備 

In [89]:
# 必要ライブラリをimport

%matplotlib inline
import sys
import igo
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import spatial
from PIL import Image
from IPython.display import display
from IPython.display import Image, HTML

pd.set_option('display.max_colwidth', -1)

In [90]:
# word2vec が実装されている　gensim というライブラリを用いる
from gensim import corpora, models, similarities
from gensim.models import word2vec as w2v

グノシーから取得した記事データをPandasのDataFrameに読み込みます

In [91]:
df_articles = pd.read_csv('csv/item_gunosy.csv')

データを確認すると、記事の
- title: タイトル
- subcategory: サブカテゴリー
- thumbnail_url: サムネイルのURL
- url: 記事のURL
が含まれていることが確認できます。また、今回の記事データは一部サムネイルURLがない記事も存在します

In [92]:
df_articles.head(1)

,_type,subcategory,thumbnail_url,title,url
0,GunosynewsItem,ゲーム,images.gunosy.com/6/11/abf91413c25e9329d0a334410d8d007e_large.jpg,ワイワイ系の懐かしトイが超絶進化！「東京おもちゃショー」で見つけた傑作選,https://gunosy.com/articles/Ru6Ac


###  3. 類似度計算を実施するためのデータの整形

gensimのword2vecに記事データを入力するためには、データを整形する必要があります

### 3-1. 日本語文字列の分かち書き

日本語の文章をword2vecによってベクトル化するためには、まず文章を単語に分ける必要があります<br>
今回はMecabではなく、[igo-python](https://pypi.python.org/pypi/igo-python/)を用いて分かち書きを行います。
また、辞書は[mecab-ipadic-neologd](https://github.com/neologd/mecab-ipadic-neologd)を利用します

In [93]:
tagger = igo.tagger.Tagger('dic/neologd/')

def parser(text):
    """ 　テキストを構文解析する関数 """
    return tagger.parse(text)

def extract_target_morphs(sentence):
    """対象の形態素をリストで返却する関数"""
    morphs = []
    # タイトル文を構文解析する
    for morph in parser(sentence):
        # 構文解析結果が空であった場合スキップする
        if not morph.surface.strip():
            continue
        lemma = morph.surface
        data = morph.feature.split(',')
        if data[0] in ['動詞', '名詞']or (data[0] == '形容詞' and data[6] != '*'):
            morphs.append(data[6])
    return morphs

助詞や記号などは文章内で多く出現し、それほど意味を持たないため除くことが多いです(これらのワードをストップワードと呼びます)<br>
今回も動詞・名詞・形容詞に絞って分かち書きを行います

In [94]:
extract_target_morphs('困惑】「てんや」の謎の新メニュー『豚角煮天丼』、普通にウマい')

['困惑', 'てんや', '謎', 'メニュー', '豚', '角', '煮る', '天丼', '普通に', 'ウマ', 'いる']

記事タイトルを分かち書きする
さらに、word2vec に入力するために構文解析(分かち書き)済みのタイトルをスペースで分割する 

In [95]:
# 記事タイトルを分かち書きする
df_articles['parsed_title'] = df_articles['title'].apply(extract_target_morphs)
df_articles['parsed_title'] = df_articles['parsed_title'].apply(lambda x:  [' '.join(v for v in x)][0])

In [96]:
df_articles[['title', 'subcategory', 'parsed_title']].head(1)

,title,subcategory,parsed_title
0,ワイワイ系の懐かしトイが超絶進化！「東京おもちゃショー」で見つけた傑作選,ゲーム,ワイ ワイ 系 懐かしい トイ 超絶 進化 東京おもちゃショー 見つける 傑作 選


分かち書き済みのタイトル文字群を一旦txtファイルに保存します

In [97]:
with open('data/titles.txt', 'w') as f:
    df_articles['parsed_title'].to_csv(f, header=None, index=None, sep=',')

## 4. word2vecによるモデルの生成と類似度計算

今回はword2vecのモデルの一つである、「Skip-gram」を用いて単語間の類似度計算を考えてみます。「Skip-gram」とは、「ある単語が入力された時に、その単語の周辺にどのような単語が出現しやすいか」を予測するタスクを解くモデルになります<br>

例えば「【ソース不要！】じゅわトロ「トマトのシンプルピザ」を作ってみない？」という文章について、「ピザ」という単語に注目した時、「トマト」や「チーズ」などのビザの食材が
周辺に多く出現する可能性が高いです。「Skip-gram」はこの周辺に出やすい単語の出現確率について、指定された単語分算出することでベクトルに変換します

In [98]:
# 構文解析済みタイトルの文字群をword2vecにかける
titles = w2v.LineSentence('data/titles.txt')
model = w2v.Word2Vec(titles, size=100, window=5, min_count=1, sg=1, workers=4)

In [99]:
# ある単語に最も類似度が高い単語を返す
model.most_similar(positive='ご飯')

[('豆腐', 0.9994983673095703),
 ('面白い', 0.9994969964027405),
 ('朝', 0.9994852542877197),
 ('運命', 0.9994773268699646),
 ('似合う', 0.999453604221344),
 ('忙しい', 0.9994499683380127),
 ('笑', 0.9994399547576904),
 ('真ん中', 0.9994328022003174),
 ('直伝', 0.9994291067123413),
 ('火', 0.9994194507598877)]

今回はトレーニングデータが少ないことから、精度があまりよくはありません。。<br>
そこで、株式会社白ヤギコーポーレーションから学習済みword2vecモデルが公開されているため、そのモデルを使用してみます<br>[word2vecの学習済み日本語モデルを公開します](http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/)

In [100]:
# 白ヤギコーポレーションから公開されている学習済みのモデルをロードする
model_path = 'model/word2vec.gensim.model'
model = w2v.Word2Vec.load(model_path)

In [101]:
# ある単語に最も類似度が高い単語を返す
model.most_similar(positive='ご飯')

[('タレ', 0.9213054180145264),
 ('汁', 0.917236328125),
 ('豆腐', 0.9129927754402161),
 ('鍋', 0.9093849658966064),
 ('味噌汁', 0.9085185527801514),
 ('油揚げ', 0.8977984189987183),
 ('おにぎり', 0.8972976803779602),
 ('梅干し', 0.893747866153717),
 ('汁物', 0.8927422761917114),
 ('天ぷら', 0.8925649523735046)]

In [102]:
# 各単語に指定したサイズのベクトルが計算されている(size=50)
model.wv['ラーメン']

array([-0.11716078, -0.07515609,  0.00853589, -0.08632582,  0.09497195,
        0.13244595, -0.13451752, -0.08669069,  0.11902761,  0.04120288,
        0.20926368,  0.1081875 , -0.06735694,  0.37801969, -0.11764119,
        0.18195644,  0.09917535, -0.11339456, -0.07504001,  0.1889106 ,
       -0.03481169, -0.079282  ,  0.09465183,  0.14899369, -0.12081202,
       -0.23041828,  0.08662395,  0.32221848, -0.08037522,  0.14042863,
        0.08189129,  0.04125   ,  0.12892513,  0.05284416,  0.04760501,
        0.04229622, -0.0771471 ,  0.03095818,  0.08163084, -0.24840046,
       -0.07823882, -0.09401023, -0.09574612,  0.06188807, -0.21567668,
        0.13368349, -0.00879957,  0.25245234,  0.18297784, -0.22478935], dtype=float32)

各単語に上記のような50次元の単語ベクトルが割り当てられました。これらの単語ベクトルを平均化することで、文章間での類似度も計算することが可能となります！

単語ベクトルの平均値から文章間での類似度を計算してみましょう

In [114]:
class MeanEmbeddingVector(object):
    def __init__(self, df, title):
        self.df = df
        self.title = title
        self.target_article_vector = None
        
    def path_to_image_html(self, path):
        return '<img src="http://'+ path + '"/>'
    
    def mean_embedding_vector(self, words):
        """ 文章内の単語ベクトルの平均値を算出する """
        featureVec = np.zeros((model.vector_size,), dtype="float32")
        nwords = 0
        # もしタイトル内の単語がモデルのボキャブラリーに含まれていた場合、その単語のベクトルをlistに追加する
        for word in words:
            if word in model.wv.vocab:
                nwords += 1
                featureVec = np.add(featureVec, model.wv[word])
        # ボキャブラリーに含まれていた単語数で割ることで、単語ベクトルの平均を算出
        if(nwords>0):
             featureVec = np.divide(featureVec, nwords)
        return list(set(featureVec))

    def cosine_similarity(self, other_article_vector):
        """ cos類似度を計算する """
        return 1 - spatial.distance.cosine(self.target_article_vector, other_article_vector)

    def similer_titles_by_mean_embedding_vector(self, df_other_article):
        """ 入力されたタイトルに近しい記事を出力する"""
        # 比較対象元の単語ベクトル平均と比較対象の単語ベクトル平均のcos類似度を計算する
        df_other_article['score'] = df_other_article['mean_vector'].apply(self.cosine_similarity)
        
        # cos類似度が高い順にソートする
        return df_other_article.sort_values('score', ascending=False).reset_index().head(5)

    def plot_similer_articles(self):
        """ 入力されたタイトルに近しい記事を計算し、可視化する"""
        self.df['thumbnail'] = self.df['thumbnail_url'].apply(self.path_to_image_html)
        self.df['mean_vector'] = self.df['parsed_title'].apply(self.mean_embedding_vector)
        
        # 比較元の単語ベクトルの平均値を代入する
        df_target_article = self.df[self.df['title'] == self.title].reset_index()
        df_target_article['score'] = None
        
        # 入力されたタイトルが記事データ内にない場合、新たにベクトルを生成し類似度を計算する
        if df_target_article.empty:
            parsed_title = extract_target_morphs(self.title)
            self.target_article_vector = self.mean_embedding_vector(parsed_title)
            df_similer_articles = self.similer_titles_by_mean_embedding_vector(df_other_article = self.df)
            print('比較対象元のタイトル: {}'.format(self.title))
            return HTML(df_similer_articles[['title', 'thumbnail', 'score', 'parsed_title']].to_html(escape=False))
        
        self.target_article_vector = df_target_article['mean_vector'][0]
        df_similer_articles = self.similer_titles_by_mean_embedding_vector(df_other_article = self.df[self.df['title'] != self.title].reset_index(drop=True))
        df_similer_articles = pd.concat([df_target_article, df_similer_articles], axis=0)
        return HTML(df_similer_articles[['title', 'thumbnail', 'score', 'parsed_title']].to_html(escape=False))

In [116]:
MeanEmbeddingVector(df=df_articles, title='ラーメン').plot_similer_articles()

比較対象元のタイトル: ラーメン


,title,thumbnail,score,parsed_title
0,【食卓の人気者に！】「鶏肉のケチャップ醤油焼き」がやみつきの旨さ,,0.547929,食卓 人気 者 鶏肉 ケチャップ 醤油 焼き 病み付き 旨い さ
1,松屋が「大創業祭」 「牛めし」50円引き,,0.547368,松屋 創業 祭 牛めし 50円 引き
2,日本で発明されたあの食べ物が、とにもかくにも大好きな中国の人たち=中国メディア,,0.526601,日本 発明 する れる 食べ物 大好き 中国 人達 中国メディア
3,セ・リーグ個人別本塁打成績 (9日現在),,0.522266,セ・リーグ 個人 別 本塁打 成績 9日 現在
4,【丸亀製麺】6月6日から3日間『牛とろ玉うどん』が半額の340円に！ これ本当に美味しいやつですよ【夜なきうどん】,,0.519418,丸亀製麺 6月6日 3日 間 牛 吐露 玉 うどん 半額 * 円 これ 美味しい やつ ですよ。 夜 ない うどん


このようにword2vecは単語間の類似度を算出することができ、さらに各単語のベクトルの平均値などを用いることで文章間の類似度を計算することが可能です。しかし、単語の並び順が考慮されないという欠点があります。<br>
そこで、文章間の類似度を計算するためにword2vecを拡張したdoc2vecという手法が提案されています。doc2vecは単語の並び順も考慮した文章間の類似度計算を実施することが可能です。

## 5.doc2vecによるモデルの生成と類似度計算

doc2vecもword2vecと同様、予め文章を分かち分けし単語に分けたデータを入力します

In [120]:
# doc2vec用のライブラリをimportする
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [121]:
#１文書ずつ、単語に分割してリストに入れていく
#words：文書に含まれる単語のリスト（単語の重複あり）
# tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）
sentences = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(open('data/titles.txt','r'))]

In [122]:
#[([単語1,単語2,単語3],文書id),...]というような行が並んでいます
sentences[:2]

[TaggedDocument(words=['ワイ', 'ワイ', '系', '懐かしい', 'トイ', '超絶', '進化', '東京おもちゃショー', '見つける', '傑作', '選'], tags=[0]),
 TaggedDocument(words=['タモリ', '黄金', 'シャチホコ', '前'], tags=[1])]

モデルのパラメータは以下についてです
- size: ベクトル化した際の次元数
- alpha: 学習率　(低いほど収束が早いですが、精度が落ちてしまいます)
- sample: 単語を無視する際の頻度の閾値 (多くの文章に登場する単語はそれほど重要ではない可能性があるため、このパラーメータで閾値を設定します)
- min_count: 学習に使う単語の最低出現回数 (sampleとは逆に出現頻度が非常に少ない単語についてもそれほど重要ではない可能性があるため、このパラメータで設定します)
- workers: 学習時のスレッド数

In [123]:
model_doc = models.Doc2Vec(dm=0, size=300, window=15, alpha=.025, min_alpha=.025, min_count=1, sample=1e-6, workers=4)
model_doc.build_vocab(sentences)

実際に記事タイトルから文章間の類似度計算を実験してみましょう

In [137]:
def path_to_image_html(path):
    return '<img src="http://'+ path + '"/>'

def similer_titles(df, title):
    """ 入力されたタイトルに近しい記事を出力する"""
    # 分かち書きされたタイトルの単語群から新しいベクトルを生成する
    new_vector = model_doc.infer_vector(title)
    # ベクトルが近いタイトルのidを取得する
    sims = model_doc.docvecs.most_similar([new_vector])
    # 類似度が高いタイトルに絞る
    df_similer_articles = df.loc[[x[0] for x in sims], :]
    # Scoreも入力しておく
    df_similer_articles['score'] = [x[1] for x in sims]
    return df_similer_articles.head(5)

def plot_similer_articles(df, title):
    """ 入力されたタイトルに近しい記事を計算し、可視化する"""
    df['thumbnail'] = df['thumbnail_url'].apply(path_to_image_html)
    target = df[df['title'] == title].reset_index()
    if target.empty:
        parsed_title = extract_target_morphs(title)
        df_similer_articles = similer_titles(df=df, title=parsed_title)
        print('比較元文章: {}'.format(title))
        return HTML(df_similer_articles[['title', 'thumbnail', 'score', 'parsed_title']].to_html(escape=False))
    
    target['score'] = None
    df_similer_articles = similer_titles(df=df[df['title'] != title], title=target['parsed_title'])
    df_similer_articles = pd.concat([target, df_similer_articles], axis=0)
    return HTML(df_similer_articles[['title', 'thumbnail', 'score', 'parsed_title']].to_html(escape=False))

今回は学習データが少ないため、word2vecと比べ精度は非常に悪いですね

In [141]:
plot_similer_articles(df_articles, 'ラーメン食べたいな')

比較元文章: ラーメン食べたいな


,title,thumbnail,score,parsed_title
1068,関門海峡を隔てて伝える戦と信仰の歴史~下関・門司~,,0.226589,関門海峡 隔てる 伝える 戦 信仰 歴史 下関 門司
13229,韓国大統領特使への安倍首相のもてなしがひどい！韓国ネットユーザーが椅子に注目し指摘=「ミスにしてはあからさま」「韓国を相当見下してるね」,,0.210238,韓国大統領 特使 安倍首相 もてなし ひどい 韓国 ネットユーザー 椅子 注目 する 指摘 ミス 仕手 あからさま 韓国 見下す てる
3625,肩こりも頭痛も「うつ伏せで腰を揺らす」だけで改善するってホント？,,0.206390,肩こり 頭痛 うつ 伏せ 腰 揺らす 改善 する ホント
6114,済州戦の騒動で約220万円の罰金...浦和が声明「今後の対応は改めて報告」,,0.203589,済州 戦 騒動 * 万 円 罰金 浦和 声明 今後 対応 報告
420,初夏のムードにピッタリのジューシーな”シトラスカラーネイル”,,0.202512,初夏 ムード ジューシー *
